### All three points were answered within the same notebook.

In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url)

soup = BeautifulSoup(html, 'lxml')

print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XnJWpApAIHkAABsKSIwAAAAE","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":946188405,"wgRevisionId":946188405,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Article

In [2]:
# We filter only for the cells within the scraped page 

cells = soup.findAll('td')

# Define a function to separate information in each cell to later be used in dataframe

def process_cell(cell):
    maybe_title = cell.findAll('b')
    title = maybe_title[0].contents[0] if len(maybe_title) >=1 else ""
    links = cell.findAll('a')
    borough = links[0]['title'] if len(links) >= 1 else ""
    neighborhood = links[1]['title'] if len(links) >= 2 else ""
    return title, borough, neighborhood

# Create lists with the data of each line that was separated

PostalCode = []
Borough = []
Neighborhood = []
for cell in cells:
    title, borough, neighborhood = process_cell(cell)
    PostalCode.append(title)
    Borough.append(borough)
    Neighborhood.append(neighborhood)
    print (title)
    print (borough)
    print (neighborhood)

M1A


M2A


M3A
North York
Parkwoods
M4A
North York
Victoria Village
M5A
Downtown Toronto
Regent Park
M6A
North York
Lawrence Manor
M7A
Queen's Park (Toronto)
Ontario Provincial Government
M8A


M9A
Etobicoke
Islington Avenue
M1B
Scarborough, Toronto
Malvern, Toronto
M2B


M3B
North York
Don Mills
M4B
East York
Parkview Hill
M5B
Downtown Toronto
Garden District, Toronto
M6B
North York

M7B


M8B


M9B
Etobicoke
West Deane Park
M1C
Scarborough, Toronto
Rouge Hill
M2C


M3C
North York
Don Mills
M4C
East York
Woodbine Heights
M5C
Downtown Toronto
St. James Town
M6C
York, Toronto
Humewood-Cedarvale
M7C


M8C


M9C
Etobicoke
Markland Wood
M1E
Scarborough, Toronto
Guildwood
M2E


M3E


M4E
The Beaches

M5E
Downtown Toronto

M6E
York, Toronto
Caledonia-Fairbanks
M7E


M8E


M9E


M1G
Scarborough, Toronto
Woburn, Toronto
M2G


M3G


M4G
East York
Leaside
M5G
Downtown Toronto
Bay Street
M6G
Downtown Toronto

M7G


M8G


M9G


M1H
Scarborough, Toronto
Woburn, Toronto
M2H
North York
Hillcrest Vil

KeyError: 'title'

In [3]:
# Create dataframe, concatenate all three columns and rename them

title = str(title)
df = pd.DataFrame([PostalCode,Borough, Neighborhood]).T
df = df.rename(columns={0:'PostalCode',1:'Borough',2:'Neighborhood'})
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,,
1,M2A,,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park (Toronto),Ontario Provincial Government
7,M8A,,
8,M9A,Etobicoke,Islington Avenue
9,M1B,"Scarborough, Toronto","Malvern, Toronto"


In [4]:
# Drop rows with unassigned Boroughs and fill Neighborhoods with their Borough

import numpy as np
df.replace('', np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace=True)
df['Neighborhood'] = df['Neighborhood'].fillna(df['Borough'])
df = df.groupby(['PostalCode','Borough'], sort=False).agg( ', '.join)
df = df.reset_index()
df.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,Lawrence Manor
4,M7A,Queen's Park (Toronto),Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,"Scarborough, Toronto","Malvern, Toronto"
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill
9,M5B,Downtown Toronto,"Garden District, Toronto"


#### DataFrame is ready: postal codes, boroughs and neighborhoods were scraped. Rows with unassigned boroughs were dropped and empty neighborhoods were filled with its respective borough. 

In [5]:
df.shape

(101, 3)

### We carry on with the second part of the exercise.

We will work with the .csv file provided.

In [6]:
coords = pd.read_csv('C:/Users/Joaquin/Documents/Cursos/IBM Data Science/9 capstone/Geospatial_Coordinates.csv')
coords.columns= ['PostalCode', 'Latitude','Longitude']
coords.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df = df.merge(coords, on = 'PostalCode')
df.shape

(101, 5)

#### We advance to the third part: cluster and analyse the data by neighborhood.

In [8]:
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [9]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
Tlat = location.latitude
Tlong = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(Tlat, Tlong))

Solving environment: ...working... done

# All requested packages already installed.

The geograpical coordinates of Toronto are 43.653963, -79.387207.


We map the different neighborhoods in Toronto to visualize the data.

In [10]:
Tmap = folium.Map(location=[Tlat, Tlong], zoom_start=11)

for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Tmap)  
    
Tmap

In [11]:
CLIENT_ID = '3ZCMZFGY0RYTXXYDUAPWEEBRKC5S2UQMUC2KOWJG0UDYRS0K' 
CLIENT_SECRET = 'G22Y5ALU1ME45OARPAOTOL3K3E4QBZIA0XB1FW0RFVBRL5YV'
VERSION = '20180605'
radius=500
LIMIT=100

In [12]:
import requests

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
tvenues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'])

Parkwoods
Victoria Village
Regent Park
Lawrence Manor
Ontario Provincial Government
Islington Avenue
Malvern, Toronto
Don Mills
Parkview Hill
Garden District, Toronto
North York
West Deane Park
Rouge Hill
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Markland Wood
Guildwood
The Beaches
Downtown Toronto
Caledonia-Fairbanks
Woburn, Toronto
Leaside
Bay Street
Downtown Toronto
Woburn, Toronto
Hillcrest Village
Bathurst Manor
Thorncliffe Park
Richmond Street
Dovercourt Village
Scarborough Village
Henry Farm
York University Heights
The Danforth
Harbourfront (Toronto)
Trinity–Bellwoods
Kennedy Park, Toronto
Bayview Village
Downsview
Riverdale, Toronto
Toronto Dominion Centre
Parkdale Village
Golden Mile, Toronto
York Mills
Downsview
The Beaches
Commerce Court
Maple Leaf, Toronto
Humber Summit
Cliffside, Toronto
Newtonbrook
Downsview
Bedford Park, Toronto
Mount Dennis
Humberlea
Birch Cliff
Willowdale, Toronto
Downsview
Lawrence Park, Toronto
Runnymede, Toronto
Weston, Toronto
Do

In [14]:
tvenues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Agincourt, Toronto",5,5,5,5,5,5
"Alderwood, Toronto",11,11,11,11,11,11
Bathurst Manor,21,21,21,21,21,21
Bay Street,82,82,82,82,82,82
Bayview Village,4,4,4,4,4,4
"Bedford Park, Toronto",25,25,25,25,25,25
Birch Cliff,4,4,4,4,4,4
Business reply mail,16,16,16,16,16,16
CN Tower,18,18,18,18,18,18


#### Neighborhood Analysis

In [15]:
# We do one hot encoding for more accurate analysis
toh = pd.get_dummies(tvenues[['Venue Category']], prefix="", prefix_sep="")

toh['Neighborhood'] = tvenues['Neighborhood'] 

fixed_columns = [toh.columns[-1]] + list(toh.columns[:-1])
toh.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# We group data by neighborhood mean frequency
tgrouped = toh.groupby('Neighborhood').mean().reset_index()
tgrouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Agincourt, Toronto",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Alderwood, Toronto",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bathurst Manor,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Bay Street,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,...,0.012195,0.000000,0.000000,0.000000,0.000000,0.012195,0.000000,0.000000,0.000000,0.012195
4,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Bedford Park, Toronto",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Birch Cliff,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Business reply mail,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500
8,CN Tower,0.0,0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Caledonia-Fairbanks,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000


#### Which are the top 5 most common venues in each neighborhood?

In [17]:
num_top_venues = 5

for hood in tgrouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tgrouped[tgrouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt, Toronto----
                       venue  freq
0             Clothing Store   0.2
1             Breakfast Spot   0.2
2                     Lounge   0.2
3               Skating Rink   0.2
4  Latin American Restaurant   0.2


----Alderwood, Toronto----
                venue  freq
0         Pizza Place  0.18
1                Pool  0.09
2      Sandwich Place  0.09
3  Athletics & Sports  0.09
4        Dance Studio  0.09


----Bathurst Manor----
           venue  freq
0           Bank  0.10
1    Coffee Shop  0.10
2       Pharmacy  0.05
3      Gift Shop  0.05
4  Shopping Mall  0.05


----Bay Street----
                       venue  freq
0                Coffee Shop  0.17
1         Italian Restaurant  0.05
2               Burger Joint  0.04
3  Middle Eastern Restaurant  0.04
4             Ice Cream Shop  0.04


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4

                      venue  freq
0              Intersection  0.15
1               Coffee Shop  0.15
2                     Hotel  0.15
3                       Gym  0.08
4  Mediterranean Restaurant  0.08


----Mount Dennis----
                venue  freq
0  Turkish Restaurant   0.2
1          Restaurant   0.2
2      Sandwich Place   0.2
3        Skating Rink   0.2
4         Coffee Shop   0.2


----New Toronto----
                venue  freq
0              Bakery  0.08
1  Seafood Restaurant  0.08
2            Pharmacy  0.08
3           Pet Store  0.08
4                Café  0.08


----North Midtown, Toronto----
                           venue  freq
0                 Sandwich Place  0.14
1                           Café  0.14
2                    Coffee Shop  0.09
3                       Pharmacy  0.05
4  Vegetarian / Vegan Restaurant  0.05


----North Toronto----
                 venue  freq
0          Coffee Shop  0.10
1       Clothing Store  0.10
2          Yoga Studio  0.05
3  Sport

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# We create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tgrouped['Neighborhood']

for ind in np.arange(tgrouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tgrouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Agincourt, Toronto",Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood, Toronto",Pizza Place,Coffee Shop,Athletics & Sports,Dance Studio,Pharmacy,Pool,Pub,Sandwich Place,Skating Rink,Gym
2,Bathurst Manor,Bank,Coffee Shop,Pharmacy,Supermarket,Deli / Bodega,Middle Eastern Restaurant,Ice Cream Shop,Shopping Mall,Restaurant,Fried Chicken Joint
3,Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Middle Eastern Restaurant,Burger Joint,Ice Cream Shop,Japanese Restaurant,Thai Restaurant,Salad Place,Gym / Fitness Center
4,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


### Clustering analysis

In [20]:
# We cluster data into 5 neighborhoods
kclusters = 5

tclusters = tgrouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tclusters)

kmeans.labels_

array([4, 0, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4,
       4, 4, 4, 4, 4, 4, 3, 0, 4, 4, 4, 4, 0, 4, 4, 4, 3, 4, 4, 1, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 0, 1, 4, 4, 4, 4, 4, 1, 4, 4, 4, 0, 4, 4,
       1, 4, 0, 4, 1, 4, 2, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 1, 4])

We create dataframe with top 10 venues per neighborhood.

In [23]:
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

df = df.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,4.0,Hockey Arena,French Restaurant,Coffee Shop,Portuguese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,4.0,Coffee Shop,Pub,Bakery,Park,Theater,Restaurant,Café,Mexican Restaurant,Yoga Studio,French Restaurant
3,M6A,North York,Lawrence Manor,43.718518,-79.464763,4.0,Clothing Store,Furniture / Home Store,Accessories Store,Athletics & Sports,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store,Vietnamese Restaurant
4,M7A,Queen's Park (Toronto),Ontario Provincial Government,43.662301,-79.389494,4.0,Coffee Shop,Park,Yoga Studio,Distribution Center,Bar,Beer Bar,Italian Restaurant,Juice Bar,Boutique,Seafood Restaurant


In [28]:
# Data formatting and cleaning
df = df.dropna()
df['Cluster_Labels'] = df.Cluster_Labels.astype(int)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Park,Food & Drink Shop,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,4,Hockey Arena,French Restaurant,Coffee Shop,Portuguese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,4,Coffee Shop,Pub,Bakery,Park,Theater,Restaurant,Café,Mexican Restaurant,Yoga Studio,French Restaurant
3,M6A,North York,Lawrence Manor,43.718518,-79.464763,4,Clothing Store,Furniture / Home Store,Accessories Store,Athletics & Sports,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store,Vietnamese Restaurant
4,M7A,Queen's Park (Toronto),Ontario Provincial Government,43.662301,-79.389494,4,Coffee Shop,Park,Yoga Studio,Distribution Center,Bar,Beer Bar,Italian Restaurant,Juice Bar,Boutique,Seafood Restaurant


In [30]:
# create map
map_clusters = folium.Map(location=[Tlat, Tlong], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighborhood'], df['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### We study clusters separately.

In [41]:
cluster0 = df.loc[df['Cluster_Labels'] == 0, df.columns[[1] + list(range(5, df.shape[1]))]]
cluster0

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,East York,0,Pizza Place,Gym / Fitness Center,Gastropub,Pharmacy,Bus Line,Breakfast Spot,Fast Food Restaurant,Intersection,Bank,Athletics & Sports
50,North York,0,Pizza Place,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
68,Etobicoke,0,Pizza Place,Middle Eastern Restaurant,Sandwich Place,Coffee Shop,Discount Store,Chinese Restaurant,Dim Sum Restaurant,Diner,Distribution Center,Dog Run
75,Etobicoke,0,Pizza Place,Bus Line,Sandwich Place,Mobile Phone Shop,Dog Run,Diner,Discount Store,Distribution Center,Doner Restaurant,College Gym
80,"Scarborough, Toronto",0,Pizza Place,Pharmacy,Noodle House,Fast Food Restaurant,Italian Restaurant,Bank,Thai Restaurant,Chinese Restaurant,Gas Station,Fried Chicken Joint
87,Etobicoke,0,Pizza Place,Grocery Store,Pharmacy,Beer Store,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore
91,Etobicoke,0,Pizza Place,Coffee Shop,Athletics & Sports,Dance Studio,Pharmacy,Pool,Pub,Sandwich Place,Skating Rink,Gym


Cluster0 is dominated by pizza places.

In [42]:
cluster1 = df.loc[df['Cluster_Labels'] == 1, df.columns[[1] + list(range(5, df.shape[1]))]]
cluster1

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Park,Food & Drink Shop,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
21,"York, Toronto",1,Park,Women's Store,Market,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant
35,East York,1,Convenience Store,Park,Metro Station,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
45,North York,1,Park,Bank,Convenience Store,Cafeteria,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
64,North York,1,Park,Bank,Convenience Store,Cafeteria,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
81,"Moore Park, Toronto",1,Park,Trail,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
83,"Scarborough, Toronto",1,Playground,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
89,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


Cluster1 is predominantly a park area.

In [43]:
cluster2 = df.loc[df['Cluster_Labels'] == 2, df.columns[[1] + list(range(5, df.shape[1]))]]
cluster2

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,Etobicoke,2,River,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Department Store


Cluster2 consists only of the river-side borough Etobicoke.

In [44]:
cluster3 = df.loc[df['Cluster_Labels'] == 3, df.columns[[1] + list(range(5, df.shape[1]))]]
cluster3

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"Scarborough, Toronto",3,Fast Food Restaurant,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
11,Etobicoke,3,Golf Course,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
27,North York,3,Fast Food Restaurant,Golf Course,Mediterranean Restaurant,Pool,Dog Run,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center


Cluster3 holds mostly fast food restaurants as the preferent places.

In [45]:
cluster4 = df.loc[df['Cluster_Labels'] == 4, df.columns[[1] + list(range(5, df.shape[1]))]]
cluster4

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,4,Hockey Arena,French Restaurant,Coffee Shop,Portuguese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,Downtown Toronto,4,Coffee Shop,Pub,Bakery,Park,Theater,Restaurant,Café,Mexican Restaurant,Yoga Studio,French Restaurant
3,North York,4,Clothing Store,Furniture / Home Store,Accessories Store,Athletics & Sports,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store,Vietnamese Restaurant
4,Queen's Park (Toronto),4,Coffee Shop,Park,Yoga Studio,Distribution Center,Bar,Beer Bar,Italian Restaurant,Juice Bar,Boutique,Seafood Restaurant
7,North York,4,Restaurant,Gym,Café,Japanese Restaurant,Coffee Shop,Beer Store,Asian Restaurant,Sporting Goods Shop,Baseball Field,Discount Store
9,Downtown Toronto,4,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Middle Eastern Restaurant,Ramen Restaurant,Restaurant,Bookstore,Diner
10,North York,4,Pub,Pizza Place,Park,Asian Restaurant,Japanese Restaurant,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run
12,"Scarborough, Toronto",4,History Museum,Bar,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
13,North York,4,Restaurant,Gym,Café,Japanese Restaurant,Coffee Shop,Beer Store,Asian Restaurant,Sporting Goods Shop,Baseball Field,Discount Store
14,East York,4,Pharmacy,Park,Spa,Beer Store,Skating Rink,Diner,Video Store,Curling Ice,Cosmetics Shop,Doner Restaurant


Cluster4 coffee shops dominating the preferences.